In [13]:
from Q_learning import *
import pickle


Word_set = cache(total_words=100)

class Hangman2:
    def __init__(self, word):
        self.word = word
        self.guessed_word = ["_"] * len(self.word)
        self.lives = 5
        self.guessed_letters = []

    def __str__(self):
        return " ".join(self.guessed_word)

    def guess(self, letter):
        if len(letter) != 1 or not letter.isalpha():
            return "Invalid input"

        letter = letter.lower()
        if letter in self.guessed_letters:
            return "Already guessed"

        self.guessed_letters.append(letter)
        if letter in self.word:
            for i, char in enumerate(self.word):
                if char == letter:
                    self.guessed_word[i] = letter
            return "Correct"
        else:
            self.lives -= 1
            return "Incorrect"

    def is_game_over(self):
        if "_" not in self.guessed_word:
            return "win"
        elif self.lives <= 0:
            return "lose"
        return None

    def get_state(self):
        return (self.guessed_word.copy(), self.guessed_letters.copy())


def evaluate_agent2(agent, games=10, delay=0, word_set=Word_set):
    if word_set is None or len(word_set) < games:
        raise ValueError("Word set must contain at least as many words as games to play.")

    wins = 0
    total_lives_left = 0

    # Disable exploration for evaluation
    original_exploration = agent.exploration_rate
    agent.exploration_rate = 0

    # Verify Q-table is loaded
    if not agent.q_table:
        print("⚠️ WARNING: Q-table is empty!")
        return

    for game in range(games):
        word = word_set[game]
        hangman = Hangman2(word)

        while True:
            state = hangman.get_state()
            normalized_state = agent.normalize_state(state)

            action = agent.choose_action(normalized_state, hangman.guessed_letters)

            if action is None:
                break  # No valid actions left

            _ = hangman.guess(action)  # ← This was missing!

            game_status = hangman.is_game_over()
            if game_status:
                if "win" in game_status:
                    wins += 1
                    total_lives_left += hangman.lives
                break

    # Restore original exploration rate
    agent.exploration_rate = original_exploration

    # Summary
    win_rate = wins / games
    avg_lives = total_lives_left / wins if wins > 0 else 0

    print("\n🎯 Evaluation Results:")
    print(f"Games played: {games}")
    print(f"Games won: {wins}")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print(f"Average lives left (wins only): {avg_lives:.2f}")



In [16]:
agent = Agent()


with open("q_table_huge.pkl", "rb") as f:
    agent.q_table = pickle.load(f)

evaluate_agent2(agent, games=100, delay=0.0)


🎯 Evaluation Results:
Games played: 100
Games won: 12
Win rate: 12.00%
Average lives left (wins only): 2.75


A 600 mb file of dictionary of Q-values provide not so compelling results.

Lets instead try a more traditional way of solving this

In [ ]:
def Allcache(total_words=500_000_000):
    url  = f"https://random-word-api.herokuapp.com/word?number={total_words}&length=5"

    response = requests.get(url)
    if response.status_code == 200:
        word_list = response.json() 
    return (word_list)

word_list = Allcache()
len(word_list)